<a href="https://www.kaggle.com/code/brijeshkachalia/tornadoes-landfall-analyzer?scriptVersionId=234701637" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Tornadoes Landfall Analyzer
---
This code performs a comprehensive analysis of tornado occurrences for a Kaggle competition by utilizing Google Search grounding techniques. It queries a model to assess the likelihood of future tornadoes and identifies the states most at risk. The code retrieves and displays relevant grounding data, including citations from web sources, and visualizes tornado statistics using Seaborn charts. This approach combines natural language processing with real-time data retrieval to enhance the accuracy and relevance of the analysis.

### Key APIs and Libraries
___
1. Tool.Types: Tool to support Google Search in Model

2. GenerateContentConfig : Holds all the configs when making a generate content API call

3. client.models.generate_content :  Generates content based on a query and configuration, utilizing the specified model.

4. client.chats.create() : Initializes a chat session with the specified model.

5. chat.send_message() : Sends a message to the chat model and retrieves the response.

6. Markdown(): Renders text in Markdown format for display.

7. Models: gemini-2.0-flash

### Key Functionalities:
---
**Search Grounding**: Enables the model to use real-time Google Search data to enhance the accuracy of responses. 

**Retry Mechanism**: Ensures that if grounding data is incomplete, the query is retried until valid data is obtained. 

**Footnote Generation**: Creates citations for the supported text, linking back to the original sources. 

**Data Visualization**: Utilizes Seaborn to plot charts based on tornado data, enhancing the analysis with visual representation. 


Use the API¶
Start by installing and importing the Gemini API Python SDK

In [ ]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

In [ ]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
# And now re-run the same query with search grounding enabled.
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

def query_with_grounding():
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents="What is likelyhood of having next tornadoes and in which state?",
        config=config_with_search,
    )
    return response.candidates[0]


rc = query_with_grounding()
Markdown(rc.content.parts[0].text)

In [ ]:
while not rc.grounding_metadata.grounding_supports or not rc.grounding_metadata.grounding_chunks:
    # If incomplete grounding data was returned, retry.
    rc = query_with_grounding()

chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}')

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

# Provide the Tornadoes metadata 

In [ ]:
from pprint import pprint

supports = rc.grounding_metadata.grounding_supports
for support in supports:
    pprint(support.to_json_dict())

# Tornadoes Footnotes

In [ ]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        rc.content.parts[0].text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"{i}. [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

In [ ]:
from IPython.display import display, Image, Markdown

def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())
    
        display(Markdown('----'))

# Tornadoes Stats for the year 2024

In [ ]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
)

chat = client.chats.create(model='gemini-2.0-flash')

response = chat.send_message(
    message="What were the tornadoes tallies, by top-10 state,  from Jan 2024 to Dec 2024?",
    config=config_with_search,
)

show_response(response)

# Seborn Chart (State vs Tornadoes)

In [ ]:
config_with_code = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    temperature=0.0,
)

response = chat.send_message(
    message="Plot seaborn chart with states with highest tornadoes, change color to ornage",
    config=config_with_code,
)

show_response(response)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Tornado data for 2024 (as provided earlier)
tornado_data = {
    'Texas': 169,
    'Nebraska': 131,
    'Iowa': 131,
    'Illinois': 126,
    'Missouri': 105,
    'Florida': 103,
    'Louisiana': 92,
    'Oklahoma': 91,
    'Kansas': 89,
    'Ohio': 74
}

# Convert the dictionary to a Pandas DataFrame for easier plotting
import pandas as pd
df = pd.DataFrame(list(tornado_data.items()), columns=['State', 'Tornado Count'])

# Sort the DataFrame by tornado count in descending order
df = df.sort_values(by='Tornado Count', ascending=False)

# Create the bar chart using Seaborn
plt.figure(figsize=(12, 6))  # Adjust figure size for better readability
sns.barplot(x='State', y='Tornado Count', data=df, color='orange')
plt.title('Top 10 States with Highest Tornado Counts in 2024')
plt.xlabel('State')
plt.ylabel('Tornado Count')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()